In [2]:
#Bring in modules
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import pandas as pd

In [17]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

In [19]:
# Retrieve the parent divs for all the articles
results = soup.find_all(class_='list_text')
headlines = []
articles = []

for result in results:        
        # Identify and return title of a news item
        title_div= result.find(class_='content_title')
        headline = title_div.find('a').text        
        headlines.append(headline)
        
        # Identify and return paragraph of the news item
        news_paragraph = result.find(class_='article_teaser_body').text
        articles.append(news_paragraph)
        
#Create dataframe        
news = pd.DataFrame({'headlines': headlines, 'articles': articles})
news.head()

,headlines,articles
0,NASA's InSight Detects First Likely 'Quake' on...,"While their causes are still unknown, one of t..."
1,Things Are Stacking up for NASA's Mars 2020 Sp...,"As the July 2020 launch date inches closer, th..."
2,Curiosity Tastes First Sample in 'Clay-Bearing...,This new region on Mars might reveal more abou...
3,More Testing for Mars InSight's 'Mole',After the mission's heat probe began hammering...
4,NASA Garners 7 Webby Award Nominations,Nominees include four JPL projects: the solar ...


In [6]:
## Scrape and store Mars images from JPL ##
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

In [7]:
# Capture Feature area box image url
mars = soup.find_all(class_="carousel_container")
for m in mars:
    image_url = m.find('a')['data-fancybox-href']
featured_image_url = 'https://www.jpl.nasa.gov'+image_url

In [8]:
# Scrape and store Mars Weather
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
# Retrieve page with the requests module
html = browser.html
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')


In [10]:
#Find and store mars_weather
mars_weather = soup.find(class_ = 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text
print(mars_weather)

InSight sol 141 (2019-04-20) low -98.3ºC (-144.9ºF) high -19.7ºC (-3.5ºF)
winds from the SW at 4.7 m/s (10.6 mph) gusting to 12.9 m/s (28.8 mph)
pressure at 7.40 hPapic.twitter.com/CQr1QQt3cM


In [11]:
#Scrape Mars Facts 
url = 'https://space-facts.com/mars/'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')
#Scrape table and put into a panda dataframe
table = soup.find(class_="tablepress tablepress-id-mars")
table_rows = table.find_all('tr')
headers=[]
facts = []

#Use a loop to find content of each row
for tr in table_rows:
    header = tr.find('td', class_="column-1").text
    headers.append(header)
    fact = tr.find('td', class_="column-2").text
    facts.append(fact)
   
mars_facts=pd.DataFrame({"header":headers, "fact": facts})

#Remove non-data elements
mars_facts['fact']=mars_facts['fact'].str.replace('\n','')
mars_facts.head(10)

,header,fact
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [12]:
#Scrape images from Mars Hemispheres
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')
titles = []
image_urls=[]

#Retrieve the hemisphere names
hemispheres = soup.find_all(class_='description')

for h in hemispheres:
    #find web address of high res image
    web_loc = h.find('a',class_="itemLink product-item" )['href']
    image_url = 'https://astrogeology.usgs.gov' + web_loc
    image_urls.append(image_url)
    
    #find name of hemisphere   
    title = h.find('h3').text
    titles.append(title)

In [13]:
#Go to each individual page to pull high res image
full_images=[]
for image_url in image_urls:
    browser.visit(image_url)
    
    # Retrieve page with the requests module
    html = browser.html
    soup = bs(html, 'html.parser')
    full= soup.find(class_='downloads')
    full_image = full.find('a')['href']
    full_images.append(full_image)

In [14]:
#Create one python dictionary
hemisphere_image_urls =pd.DataFrame({'title':titles, 
                                     'img_url':full_images})
hemisphere_image_urls.head()

,title,img_url
0,Cerberus Hemisphere Enhanced,http://astropedia.astrogeology.usgs.gov/downlo...
1,Schiaparelli Hemisphere Enhanced,http://astropedia.astrogeology.usgs.gov/downlo...
2,Syrtis Major Hemisphere Enhanced,http://astropedia.astrogeology.usgs.gov/downlo...
3,Valles Marineris Hemisphere Enhanced,http://astropedia.astrogeology.usgs.gov/downlo...


In [15]:
browser.quit()